# There are 3 seperate datasets that will be queried
# First up is "Water chemistry (Great Lakes nearshore areas)"
[Link](https://www.ontario.ca/data/water-chemistry-great-lakes-nearshore-areas)

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import os
import re

# globally set max columns and max rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

home = os.path.abspath("")
raw_data = os.path.join(home, "raw_data")
clean_data = os.path.join(home, "clean_data")
print(home)
print(raw_data)
print(clean_data)

In [ ]:
# pathway to file
excel_path = os.path.join(raw_data, "Great_Lakes_Chemistry", "GLIS_2000_2015_WATER_CHEMISTRY.xlsx")

In [ ]:
# get sheet names
import xlrd

xls = xlrd.open_workbook(filename=excel_path, on_demand=True)
sheet_names = xls.sheet_names()
print(sheet_names)

In [ ]:
# define a list of lakes that are in the excel file
lakes = ["huron", "erie", "ontario", "superior"]

In [ ]:
# append all dataframes to 1 dataframe using pd.concat()
dataframe_list = []

for x in range(len(sheet_names)):
    
    df = pd.read_excel(excel_path, sheet_name=sheet_names[x])
    df.insert(loc=0, column='lake', value=lakes[x])
    
    dataframe_list.append(df)
    del df
    
df_combined1 = pd.concat(dataframe_list)

df_combined1.sample(30)

In [ ]:
# get row number and column number
df_combined1.shape

In [ ]:
# rename columns, drop unwanted columns and drop any nulls
df_combined1.columns = [
    'lake', 'water_body', 'date_collect', 'sample_num', 'matrix',
    'sample_type', 'sample_descr', 'remove1',
    'remove2', 'remove3', 'station_num', 'station_descr',
    'latitude', 'longitude', 'water_depth', 'sample_depth',
    'secchi_depth', 'lims_method', 'lims_product',
    'analyte', 'test_code', 'result', 'unit',
    'remove4', 'remove5'
]

drop_columns = [
    'matrix', 'sample_type', 'sample_descr', 'remove1',
    'remove2', 'remove3', 'remove4', 'remove5',
    'water_depth', 'sample_depth', 'secchi_depth',
    'lims_method', 'lims_product', 'test_code'
]

df_combined2 = df_combined1.copy()

df_combined2.drop(drop_columns, axis=1, inplace=True)
df_combined2.dropna(inplace=True)

df_combined2["analyte"].unique()

In [ ]:
# get rid of several types of analytes, not useful for machine learning
# focus on inorganic and some organic ones only
df_clean1 = df_combined2.copy()

df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='Microcystin.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='.*microcystin.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='Perflu.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='.*chloro.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='.*fluor.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='Pyrethrin.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='.*Chlordane', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='Endosulphan.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='Heptachlor.*', value = '-', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='PCB.*', value = '-', regex = True)

df_clean1 = df_clean1[df_clean1["analyte"] != "-"]
df_clean1 = df_clean1[df_clean1["result"] != "-"]
df_clean1 = df_clean1[df_clean1["analyte"] != 'Conductivity Estimated']
df_clean1 = df_clean1[df_clean1["analyte"] != 'Chlorophyll - a; corrected']
df_clean1 = df_clean1[df_clean1["analyte"] != 'Nitrogen; nitrite']
df_clean1 = df_clean1[df_clean1["analyte"] != 'Nitrogen; total Kjeldahl']
df_clean1 = df_clean1[df_clean1["analyte"] != "Ion balance calculation"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Anions"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Cations"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Langeliers index calculation"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Anatoxin-A"]
df_clean1 = df_clean1[df_clean1["analyte"] != "pp-DDD"]
df_clean1 = df_clean1[df_clean1["analyte"] != "pp-DDE"]
df_clean1 = df_clean1[df_clean1["analyte"] != "pp-DDT"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Toxaphene"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Trifluralin"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Permethrin"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Piperonyl Butoxide"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Aldrin"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Dieldrin"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Methoxychlor"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Endrin"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Mirex"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Oxychlordane"]
df_clean1 = df_clean1[df_clean1["analyte"] != "op-DDT"]
df_clean1 = df_clean1[df_clean1["analyte"] != "Saturation pH Estimated"]

df_clean1["analyte"].unique()

In [ ]:
# change the name of several values
# will aggregate these values specifically
df_clean2 = df_clean1.copy()

df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='Solids.*', value = 'solids', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='.*ammonia.*', value = 'ammonia', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='.*nitrate.*', value = 'nitrate_ite', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='Nitrogen.*', value = 'nitrogen', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='Alkalinity.*', value = 'alkaline', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='Phosphorus.*', value = 'phosphorus', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='.* pH .*', value = 'pH', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='Carbon.*', value = 'carbon', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='Silic.*', value = 'silicon', regex = True)
df_clean2['analyte'] = df_clean2["analyte"].replace(to_replace ='Chlorophyll.*', value = 'chlorophyll', regex = True)


df_clean2["analyte"].unique()

In [ ]:
# lowercase all values in "analyte" column
# will make these values into columns later
df_clean3 = df_clean2.copy()
df_clean3['analyte'] = df_clean3['analyte'].str.lower()
df_clean3["analyte"].unique()

In [ ]:
# all value in "result" column is float
# no need to clean up any strings
np.array_equal(df_clean3.result, df_clean3.result.astype(float))

In [ ]:
# after all of these transformations view the shape of the dataframe
df_clean3.shape

In [ ]:
df_clean4 = df_clean3.copy()

# change so all units are the same
# convert the units as well
# ph as unitless numbers and turbidity is measured by FTU
# make a note of this
df_clean4.loc[df_clean4.unit == "MG/L", 'result'] *= 1000000
df_clean4.loc[df_clean4.unit == "UG/L", 'result'] *= 1000
df_clean4.loc[df_clean4.unit == "MG/L", 'unit'] = "ng/L"
df_clean4.loc[df_clean4.unit == "UG/L", 'unit'] = "ng/L"
df_clean4.loc[df_clean4.unit == "NG/L", 'unit'] = "ng/L"

df_clean4.head(1)

In [ ]:
# pivot the dataframe so that analytes become the column headers
# fix the columns from multiindex
df_pivot = df_clean4.copy()
df_pivot = pd.pivot_table(df_pivot, values=['result'],\
                          index=['lake', 'water_body', "date_collect", "sample_num",\
                                 "station_num", "station_descr", "latitude",\
                                 "longitude"], columns=['analyte'], aggfunc=np.sum)

df_pivot.reset_index(level=['lake', 'water_body', 'date_collect',\
                            'sample_num', 'station_num', 'station_descr',\
                           'latitude', 'longitude'], inplace=True)

In [ ]:
# new shape is received for the data from the pivot
# lose rows but gain columns
df_pivot.shape

In [ ]:
# fix the column headers
df_pivot.columns = df_pivot.columns.droplevel(0)
df_pivot.columns = ['lake', 'water_body', 'date_collect', 'sample_num', 'station_num',
       'station_descr', 'latitude', 'longitude', 'alkaline', 'aluminum', 'ammonia',
       'antimony', 'arsenic', 'barium', 'beryllium', 'boron', 'cadmium',
       'calcium', 'carbon', 'chloride', 'chlorophyll', 'chromium', 'cobalt',
       'conductivity', 'copper', 'hardness', 'iron', 'lead', 'magnesium',
       'manganese', 'mercury', 'molybdenum', 'nickel', 'nitrate_ite',
       'nitrogen', 'ph', 'phosphorus', 'potassium', 'selenium', 'silicon',
       'silver', 'sodium', 'solids', 'strontium', 'sulphate', 'thallium',
       'titanium', 'turbidity', 'uranium', 'vanadium', 'zinc'
]

df_pivot.head(1)

In [ ]:
# groupby by all descriptors in order to aggregate "solids", "nitrogen", "phosphorous", "alkinility",
# silicon and carbon
# the dataset split these values apart for better determination of element types
df_groupby1 = df_pivot.copy()

df_groupby1 = df_groupby1.groupby(['lake', 'water_body', 'date_collect', 'station_num',\
                                   'sample_num', 'station_descr', 'latitude', 'longitude']).sum()

df_groupby1.reset_index(level=['lake', 'water_body', 'date_collect',\
                            'sample_num', 'station_num', 'station_descr',\
                           'latitude', 'longitude'], inplace=True)

In [ ]:
# turbidity and pH are special values
# if pH = 0, that means the lake has battery acid for water, more then likely the aggregation did not perserve null values
# also any negative numbers in any of the elemental concentrations do not make any sense
# need to replace with more common sense numbers

# list of columns
less_than = [
        'alkaline', 'aluminum',
       'ammonia', 'antimony', 'arsenic', 'barium', 'beryllium', 'boron',
       'cadmium', 'calcium', 'carbon', 'chloride', 'chlorophyll', 'chromium',
       'cobalt', 'conductivity', 'copper', 'hardness', 'iron', 'lead',
       'magnesium', 'manganese', 'mercury', 'molybdenum', 'nickel',
       'nitrate_ite', 'nitrogen', 'ph', 'phosphorus', 'potassium', 'selenium',
       'silicon', 'silver', 'sodium', 'solids', 'strontium', 'sulphate',
       'thallium', 'titanium', 'turbidity', 'uranium', 'vanadium', 'zinc'
]

# replace anything less than 0 with 0
for y in less_than:
    df_groupby1.loc[df_groupby1[y] <= 0, y] = 0
    
df_groupby1.min()

In [ ]:
# replace all 0 with NaN
df_groupby1.replace(0, np.nan, inplace=True)
df_groupby1.head(1)

In [ ]:
df_groupby1.shape

In [ ]:
df_groupby1.isna().sum()/df_groupby1.shape[0]

In [ ]:
# keep these columns where the % of NaN is less than 40%, arbitrary decision
from_first = ['aluminum', 'ammonia', 'barium', 'calcium', 'carbon',
             'chloride', 'chlorophyll', 'chromium', 'conductivity',
             'copper', 'hardness', 'magnesium', 'manganese', 'mercury',
              'molybdenum', 'nitrate_ite', 'phosphorus', 'potassium',
             'silicon', 'sodium', 'strontium', 'sulphate', 'turbidity',
             'vanadium', 'zinc']

In [ ]:
# reorder columns
df_groupby1 = df_groupby1[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',
       'station_descr', 'latitude', 'longitude'] + from_first]
df_groupby1.shape

In [ ]:
# change format of "date_collect" to YYYY-MM-DD to be in line with second dataset
df_groupby1["date_collect"] = df_groupby1["date_collect"].dt.strftime("%Y-%m-%d")
df_groupby1.head(1)

In [ ]:
df_groupby1.shape

In [ ]:
# drop rows if the following rows are not populated
df_groupby1.dropna(subset=from_first, thresh=1, inplace=True)
df_groupby1.shape

In [ ]:
# create dataframes
first_dataset = df_groupby1[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',
       'station_descr', 'latitude', 'longitude', 'conductivity', 'hardness', 'turbidity',
        'chlorophyll', 'ammonia', 'nitrate_ite', 'aluminum', 'barium', 'calcium', 'carbon',
        'chloride', 'chromium', 'copper', 'magnesium', 'manganese', 'mercury', 'molybdenum',
        'phosphorus', 'potassium', 'silicon', 'sodium', 'strontium', 'sulphate', 'vanadium', 'zinc']].copy()

first_metadata_dataset = df_groupby1[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',\
       'station_descr', 'latitude', 'longitude']].copy()

# save dataframe to .csv file
first_data = os.path.join(clean_data, "first.csv")
first_metadata = os.path.join(clean_data, "first_metadata.csv")

first_dataset.to_csv(first_data)
first_metadata_dataset.to_csv(first_metadata)

# Second up is "Lake water quality at drinking water intakes"
[Link](https://www.ontario.ca/data/lake-water-quality-drinking-water-intakes)

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import os
import re

# globally set max columns and max rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

home = os.path.abspath("")
raw_data = os.path.join(home, "raw_data")
clean_data = os.path.join(home, "clean_data")
print(home)
print(raw_data)
print(clean_data)

In [ ]:
# get excel file names
path = os.path.join(raw_data, "Water_Quality_Drinking_Intakes")

files = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.xlsx' in file:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

In [ ]:
# get sheet names
import xlrd

sheet_name_dict = {}

files_index = [1, 2, 3, 5]

for x in range(len(files_index)):
    
    xls = xlrd.open_workbook(filename=files[files_index[x]], on_demand=True)
    sheet_names = {x : xls.sheet_names()}
    
    print(sheet_names)
    
    sheet_name_dict.update(sheet_names)
    
    del xls, sheet_names

In [ ]:
# append all dataframes to 1 dataframe using pd.concat()
dataframe_list_books = []

for z in range(len(files_index)):
    
    dataframe_list_sheets = []
    
    for x in range(len(sheet_name_dict[z])):

        df = pd.read_excel(files[files_index[z]], sheet_name=sheet_name_dict[z][x])

        dataframe_list_sheets.append(df)
        del df

    df_combined1 = pd.concat(dataframe_list_sheets)
    
    dataframe_list_books.append(df_combined1)

df_combined2 = pd.concat(dataframe_list_books)
del dataframe_list_books

df_combined2.sample(30)

In [ ]:
# get the shape of this dataframe and make a copy
df_second = df_combined2.copy()
df_second.shape

In [ ]:
df_second["TEST_DESCRIPTION"].unique()

In [ ]:
df_second["UNIT"].unique()

In [ ]:
df_second.head(1)

In [ ]:
df_second.columns

In [ ]:
# rename columns, drop unwanted columns and drop any nulls
df_second.columns = [
        'sample_num', 'result_num', 'lake', 'station_num', 'location',
        'date_collect', 'remove1', 'analyte', 'result', 'unit',
        'remove2', 'remove3'
]

drop_columns = [
    'result_num', 'remove1', 'remove2', 'remove3'
]

df_second2 = df_second.copy()

df_second2.drop(drop_columns, axis=1, inplace=True)
df_second2.dropna(inplace=True)

df_second2["analyte"].unique()

In [ ]:
# get rid of several types of analytes, not useful for machine learning
# focus on inorganic and some organic ones only
df_clean1 = df_second2.copy()

df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='CHLOROPHYLL.*', value = 'chlorophyll', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='AMMONIUM.*', value = 'ammonia', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='NITRITE.*', value = 'nitrate_ite', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='NITRATE.*', value = 'nitrate_ite', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='PHOSPH.*', value = 'phosphorus', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='CARBON.*', value = 'carbon', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='SILICAT.*', value = 'silicon', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='CONDUCTIVITY.*', value = 'conductivity', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='PH .*', value = 'ph', regex = True)
df_clean1['analyte'] = df_clean1["analyte"].replace(to_replace ='CHLORIDE.*', value = 'chloride', regex = True)

df_clean1 = df_clean1[df_clean1["analyte"] != "NITROGEN,TOT,KJELDAHL/UNF.REA"]

df_clean1["analyte"].unique()

In [ ]:
# all value in "result" column is float
# no need to clean up any strings
np.array_equal(df_clean1.result, df_clean1.result.astype(float))

In [ ]:
df_clean1.shape

In [ ]:
df_clean1["unit"].unique()

In [ ]:
df_clean1.head(1)

In [ ]:
df_clean2 = df_clean1.copy()

# change so all units are the same
# convert the units as well
# ph as unitless numbers and turbidity is measured by FTU
# make a note of this
df_clean2.loc[df_clean2.unit == "mg/L", 'result'] *= 1000000
df_clean2.loc[df_clean2.unit == "ug/L", 'result'] *= 1000
df_clean2.loc[df_clean2.unit == "mg/L", 'unit'] = "ng/L"
df_clean2.loc[df_clean2.unit == "ug/L", 'unit'] = "ng/L"

# Change "lake" column values
df_clean2.loc[df_clean2.lake == "Lake Erie", 'lake'] = "erie"
df_clean2.loc[df_clean2.lake == "Lake Huron", 'lake'] = "huron"
df_clean2.loc[df_clean2.lake == "Lake Ontario", 'lake'] = "ontario"
df_clean2.loc[df_clean2.lake == "Lake Superior", 'lake'] = "superior"

df_clean2.head(1)

In [ ]:
# pivot the dataframe so that analytes become the column headers
# fix the columns from multiindex
df_pivot = df_clean2.copy()
df_pivot = pd.pivot_table(df_pivot, values=['result'],\
                          index=['sample_num', 'lake',\
                                 'station_num', 'location', 'date_collect'],\
                          columns=['analyte'], aggfunc=np.sum)

df_pivot.reset_index(level=['sample_num', 'lake',\
                                 'station_num', 'location', 'date_collect'], inplace=True)

In [ ]:
# new shape is received for the data from the pivot
# lose rows but gain columns
df_pivot.shape

In [ ]:
df_pivot.head(1)

In [ ]:
# fix the column headers
df_pivot.columns = df_pivot.columns.droplevel(0)
df_pivot.columns = [
    'sample_num', 'lake',
    'station_num', 'location', 'date_collect',
    'ammonia', 'carbon', 'chloride', 'chlorophyll',
    'conductivity', 'nitrate_ite', 'ph', 'phosphorous', 'silicon'
]

df_pivot.head(1)

In [ ]:
# groupby by all descriptors in order to aggregate "ammonia", "nitrates and nitrites", "phosphorus",
# silicon and carbon
# the dataset split these values apart for better determination of element types
df_groupby1 = df_pivot.copy()

df_groupby1 = df_groupby1.groupby(['sample_num', 'lake',\
                                'station_num', 'location', 'date_collect']).sum()

df_groupby1.reset_index(level=['sample_num', 'lake',\
                                'station_num', 'location', 'date_collect'], inplace=True)

In [ ]:
# turbidity and pH are special values
# if pH = 0, that means the lake has battery acid for water, more then likely the aggregation did not perserve null values
# also any negative numbers in any of the elemental concentrations do not make any sense
# need to replace with more common sense numbers

# list of columns
less_than = [
        'ammonia', 'carbon', 'chloride', 'chlorophyll',
       'conductivity', 'nitrate_ite', 'ph', 'phosphorous', 'silicon'
]

# replace anything less than 0 with 0
for y in less_than:
    df_groupby1.loc[df_groupby1[y] <= 0, y] = 0
    
df_groupby1.min()

In [ ]:
# replace all 0 with NaN
df_groupby1.replace(0, np.nan, inplace=True)
df_groupby1.head(1)

In [ ]:
df_groupby1.isna().sum()/df_groupby1.shape[0]

In [ ]:
# keep these columns where the % of NaN is less than 60%, arbitrary decision
from_second = ['ammonia', 'carbon', 'chloride', 'chlorophyll',
                'conductivity', 'nitrate_ite', 'phosphorous', 'silicon']

In [ ]:
# reorder columns
df_groupby1 = df_groupby1[['lake', 'date_collect', 'station_num', 'sample_num',
       'location'] + from_second]
df_groupby1.shape

In [ ]:
# populate lists with google determined latitudes and longitudes
import requests
import json
from API_KEY.apikey import API_KEY

list_of_stations = df_groupby1["location"].unique()

lats = []
longs = []

for a in list_of_stations:

    url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={a}&inputtype=textquery&fields=name,geometry&locationbias=point:lat,lng&key={API_KEY}"

    lat_lng = requests.get(url)
    print("success")
    lat_lng = lat_lng.json()
    
    try:
    
        lats.append(lat_lng["candidates"][0]["geometry"]["location"]["lat"])
        longs.append(lat_lng["candidates"][0]["geometry"]["location"]["lng"])
        del lat_lng
        
    except:
        lats.append(None)
        longs.append(None)

In [ ]:
dict_df = {
    "location": list_of_stations,
    "latitude": lats,
    "longitude": longs
}
df_location = pd.DataFrame.from_dict(dict_df)
df_location

In [ ]:
df_groupby1.shape

In [ ]:
merge_df = pd.merge(df_location, df_groupby1, how='inner', on="location").copy()
merge_df.head(1)

In [ ]:
merge_df.shape

In [ ]:
merge_df.columns

In [ ]:
# create columns filled with NaN to match with the first dataset
merge_df["water_body"] = None
merge_df["hardness"] = None
merge_df["turbidity"] = None
merge_df["aluminum"] = None
merge_df["barium"] = None
merge_df["calcium"] = None
merge_df["chromium"] = None
merge_df["copper"] = None
merge_df["magnesium"] = None
merge_df["manganese"] = None
merge_df["mercury"] = None
merge_df["molybdenum"] = None
merge_df["potassium"] = None
merge_df["sodium"] = None
merge_df["strontium"] = None
merge_df["sulphate"] = None
merge_df["vanadium"] = None
merge_df["zinc"] = None

In [ ]:
merge_df.shape

In [ ]:
# drop rows if the following rows are not populated
merge_df.dropna(subset=from_second, thresh=1, inplace=True)
merge_df.shape

In [ ]:
# reorder columns
merge_df.rename(columns={"location": "station_descr", "phosphorous": "phosphorus"}, inplace=True)

merge_df = merge_df[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',
       'station_descr', 'latitude', 'longitude', 'conductivity', 'hardness', 'turbidity',
        'chlorophyll', 'ammonia', 'nitrate_ite', 'aluminum', 'barium', 'calcium', 'carbon',
        'chloride', 'chromium', 'copper', 'magnesium', 'manganese', 'mercury', 'molybdenum',
        'phosphorus', 'potassium', 'silicon', 'sodium', 'strontium', 'sulphate', 'vanadium', 'zinc']]
merge_df.head(1)

In [ ]:
# create dataframes
second_dataset = merge_df[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',
       'station_descr', 'latitude', 'longitude', 'conductivity', 'hardness', 'turbidity',
        'chlorophyll', 'ammonia', 'nitrate_ite', 'aluminum', 'barium', 'calcium', 'carbon',
        'chloride', 'chromium', 'copper', 'magnesium', 'manganese', 'mercury', 'molybdenum',
        'phosphorus', 'potassium', 'silicon', 'sodium', 'strontium', 'sulphate', 'vanadium', 'zinc']].copy()

second_metadata_dataset = merge_df[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',
                       'station_descr', 'latitude', 'longitude']].copy()

# save dataframe to .csv file
second_data = os.path.join(clean_data, "second.csv")
second_metadata = os.path.join(clean_data, "second_metadata.csv")

second_dataset.to_csv(second_data)
second_metadata_dataset.to_csv(second_metadata)

# Third up is "Drinking Water Surveillance Program"
[Link](https://www.ontario.ca/data/drinking-water-surveillance-program)

# Sadly this dataset does not identify which great lake is the source of water for each municipal system
# Let us now merge the above 2 datasets together

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import os
import re

# globally set max columns and max rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

home = os.path.abspath("")
raw_data = os.path.join(home, "raw_data")
clean_data = os.path.join(home, "clean_data")
print(home)
print(raw_data)
print(clean_data)

In [ ]:
first_csv = os.path.join(clean_data, "first.csv")

first_df = pd.read_csv(first_csv)
first_df.drop(["Unnamed: 0"], axis=1, inplace=True)
first_df.head(5)

In [ ]:
first_df.shape

In [ ]:
second_csv = os.path.join(clean_data, "second.csv")

second_df = pd.read_csv(second_csv)
second_df.drop(["Unnamed: 0"], axis=1, inplace=True)
second_df.head(5)

In [ ]:
second_df.shape

In [ ]:
# are the columns equal
first_df.columns == second_df.columns

In [ ]:
# concatenate the 2 dataframes
complete_df = pd.concat([first_df, second_df])
complete_df.sample(30)

In [ ]:
complete_df.shape

In [ ]:
# sort values by "lake" and "sample_num"
complete_df.sort_values(by=['lake', 'sample_num'], ascending=True, na_position='first', inplace=True)
complete_df.head(5)

In [ ]:
# create dataframes
total_dataset = complete_df[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',
       'station_descr', 'latitude', 'longitude', 'conductivity', 'hardness', 'turbidity',
        'chlorophyll', 'ammonia', 'nitrate_ite', 'aluminum', 'barium', 'calcium', 'carbon',
        'chloride', 'chromium', 'copper', 'magnesium', 'manganese', 'mercury', 'molybdenum',
        'phosphorus', 'potassium', 'silicon', 'sodium', 'strontium', 'sulphate', 'vanadium', 'zinc']].copy()

total_metadata_dataset = complete_df[['lake', 'water_body', 'date_collect', 'station_num', 'sample_num',
                       'station_descr', 'latitude', 'longitude']].copy()

total_ML_dataset = complete_df[['lake', 'conductivity', 'hardness', 'turbidity',
        'chlorophyll', 'ammonia', 'nitrate_ite', 'aluminum', 'barium', 'calcium', 'carbon',
        'chloride', 'chromium', 'copper', 'magnesium', 'manganese', 'mercury', 'molybdenum',
        'phosphorus', 'potassium', 'silicon', 'sodium', 'strontium', 'sulphate', 'vanadium', 'zinc']].copy()

# save dataframe to .csv file
total_data = os.path.join(clean_data, "total.csv")
total_metadata = os.path.join(clean_data, "total_metadata.csv")
total_ML_data = os.path.join(clean_data, "total_ML_data.csv")

total_dataset.to_csv(total_data, index=False)
total_metadata_dataset.to_csv(total_metadata, index=False)
total_ML_dataset.to_csv(total_ML_data, index=False)